In [1]:
import numpy as np

import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pickle
import psycopg2

from datetime import datetime
import time
import pprint

import os

In [2]:
os.chdir('Data')

In [3]:
date = datetime.strftime(datetime.today(), "%Y-%m-%d")

params = {
    'host': 'localhost',
    'user': 'keith',
    'password': 'Password1@'
}

conn = psycopg2.connect(**params, dbname='jobs_links')
cur = conn.cursor()

In [4]:
def get_soup(url):
    chrome_options = webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='../../../anaconda3/bin/chromedriver',
                              options=chrome_options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

In [5]:
def grab_job_links(base_url):
    
    chrome_options = webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='../../../anaconda3/bin/chromedriver',
                              options=chrome_options)
    driver.get(base_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    
    urls = []
            
    for link in soup.find_all('div', {'class': 'title'}):
        try:
            partial_url = link.a.get('href')
            url = 'https://www.indeed.com' + partial_url
            urls.append(url)
            del url
            del partial_url

        except:
            continue
    
    del soup
        
    return urls

In [6]:
def get_jobs_pages(cur):

    date = str(datetime.strftime(datetime.today(), "%Y-%m-%d"))
    
    cur.execute("SELECT query, city, link FROM job_listings_pages;")
    job_listing_urls = cur.fetchall()
    
    for query, city, job_listing_url in job_listing_urls:
        
        urls = grab_job_links(job_listing_url)
        
        values = str([(date, query, city, url) for url in urls])[1:-1]
        
        if values:
            cur.execute("INSERT INTO jobs_pages \
            (date_pulled, query, city, link) VALUES "
            + values + ";")
        
            conn.commit()
        
        time.sleep(np.random.poisson(100)/50)

In [7]:
get_jobs_pages(cur)

TimeoutException: Message: timeout
  (Session info: headless chrome=75.0.3770.100)
